# Work in Progress

# Objectives:
### To practice the application of descriptive and inferential Statistics 101 tools to COVID-19 data.
### In specific, we will look at mortality rate statistics across time and across regions in Brazil.

In [1]:
import os
import time
import pickle
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
save_path = 'data' + os.sep + 'processed' + os.sep
JanJul2020 = pd.read_pickle(f'{save_path}JanJul2020')
AugDec2020 = pd.read_pickle(f'{save_path}AugDec2020')
JanJul2021 = pd.read_pickle(f'{save_path}JanJul2021')

In [6]:
def get_value(df, place, date):
    return df.loc[place].loc[date]['Death rate']

start = time.time()
value = get_value(JanJul2021, 'Brasília', '2021-01-01')
end = time.time()
print(f"Obtained value {value} in {end - start} seconds.")

Obtained value 1413.8046767318858 in 0.00700068473815918 seconds.
